In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import numpy as np
from models.resnet import ResNet
from models.unet import UNet
from models.skip import skip
from models.dcgan import dcgan
import torch
import torch.optim
from sklearn.model_selection import train_test_split
from torchmetrics.image import StructuralSimilarityIndexMeasure
from torchmetrics.image import TotalVariation
import albumentations as A
from albumentations.pytorch import ToTensorV2
# from utils.inpainting_utils import *
import torch.nn as nn
from PIL import Image

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
dtype = torch.cuda.FloatTensor

PLOT = True
imsize = -1
dim_div_by = 64


In [2]:
import os
import sys
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
# from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
# from torchsummary import summary

from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

# # Get the directory of the current script
# current_script_dir = os.path.dirname(os.path.abspath(__file__))

# # Navigate up to the 'scripts' directory
# scripts_dir = os.path.dirname(current_script_dir)

# # Add 'scripts' directory to sys.path
# sys.path.append(scripts_dir)

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate


t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
f = r-a  # free inside reserved


16908615680
0
0


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cuda


In [4]:
net = UNet()
trained_unet = '/home/jc-merlab/Pictures/Data/trained_models/inpainting/14Dec_att_generative_was_YCB.pth'
net.load_state_dict(torch.load(trained_unet))
weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_ld_b3_e25.pth'
model = torch.load(weights_path).to(device)

# discriminator = Discriminator()
# trained_discriminator = '/home/jc-merlab/Pictures/Data/trained_models/inpainting/14Dec_att_discrimiator_was_YCB.pth'
# discriminator.load_state_dict(torch.load(trained_discriminator))

In [11]:
net.eval()
net.to(device)

image_transform = T.Compose([
    T.Resize((480, 640)),
    T.ToTensor(),
    T.Normalize(mean=[0.209555113170385, 0.22507974363977162, 0.20982026500023962], std=[0.20639409678896012, 0.19208633033458372, 0.20659148273508857]),
])


def denormalize(tensor, means, stds):
    means = torch.tensor(means).view(-1, 1, 1)
    stds = torch.tensor(stds).view(-1, 1, 1)

    if tensor.is_cuda:
        means = means.to(tensor.device)
        stds = stds.to(tensor.device)
    tensor = tensor * stds + means

    return tensor

def predict_inpaint(frame):
    # Convert frame to PIL Image to apply torchvision transforms
    frame = Image.fromarray(frame)
    frame = image_transform(frame).unsqueeze(0)  # Add batch dimension
    frame = frame.cuda()  # or .cpu() if not using GPU
    with torch.no_grad():
        prediction = net(frame)
    # Optionally, reverse normalization or other transformations here
    # Convert the tensor to a numpy array for OpenCV to display or save
    prediction = denormalize(prediction, [0.209555113170385, 0.22507974363977162, 0.20982026500023962], [0.20639409678896012, 0.19208633033458372, 0.20659148273508857])

    prediction = prediction.squeeze().cpu().numpy()
    prediction = np.transpose(prediction, (1, 2, 0))  # Change from (C, H, W) to (H, W, C)
    prediction = (prediction * 255).astype(np.uint8)
    
    # return cv2.GaussianBlur(prediction, ksize=(1,1), sigmaX=0)
    return cv2.fastNlMeansDenoisingColored(prediction, None, 5, 5)

def predict_kp(frame):
    frame_pil = Image.fromarray(frame)
    frame_tensor = F.to_tensor(frame_pil).to(device)
    frame_tensor = frame_tensor.unsqueeze(0)
#     frame = list(frame)
    with torch.no_grad():
        model.to(device)
        model.eval()
        output = model(frame_tensor)
    frame_tensor = (frame_tensor[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
    scores = output[0]['scores'].detach().cpu().numpy()
    high_scores_idxs = np.where(scores > 0.7)[0].tolist() # Indexes of boxes with scores > 0.7
    post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], \
                            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy()
    keypoints = []
    for kps in output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            keypoints.append(list(map(int, kps[0,0:2])))
            
    labels = []
    for label in output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
            labels.append(label)
    keypoints_ = [x for _,x in sorted(zip(labels,keypoints))]
#     print(keypoints_)
    return keypoints_

def ensure_uint8(image):
    if image.dtype != np.uint8:
        # Convert to [0, 255] if it's not already
        if np.max(image) <= 1:
            image = (image * 255).astype(np.uint8)
        else:
            image = image.astype(np.uint8)
    print("image is uint8")
    return image

def draw_keypoints(image, keypoints, color=(0, 255, 0), radius=5, thickness=-1):
    # Ensure the image is a NumPy array
    image = ensure_uint8(image)
    # Confirm image properties
    print(f"Image dtype: {image.dtype}, shape: {image.shape}")
    if not isinstance(image, np.ndarray):
        raise ValueError("Image must be a NumPy array")
    
    for (x, y) in keypoints:
        # Ensure keypoints are integers and within image bounds
        x, y = int(round(x)), int(round(y))
        if 0 <= x < image.shape[1] and 0 <= y < image.shape[0]:  # Check bounds
            print(type(image))
            cv2.circle(image, (x, y), radius, color, thickness)
        else:
            print(f"Skipping out-of-bounds keypoint: ({x}, {y})")
    return image

    
# Open the video
cap = cv2.VideoCapture('/home/jc-merlab/Pictures/panda_data/images_for_occlusion/1/occ_test_1.avi')
# Prepare for saving the output video (if needed)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/home/jc-merlab/Pictures/panda_data/images_for_occlusion/1/occ_test_1_kp.avi', fourcc, 20.0, (640, 480))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Process and inpaint the frame
    inpainted_frame = predict_inpaint(frame)    
    # Predict keypoints on the inpainted frame
    keypoints = predict_kp(inpainted_frame)
    
    output_frame = inpainted_frame.copy()
    
    for (x,y) in keypoints:
        x, y = int(round(x)), int(round(y))
        if 0 <= x < inpainted_frame.shape[1] and 0 <= y < inpainted_frame.shape[0]:  # Check bounds
            print(type(inpainted_frame))
            cv2.circle(output_frame, (x, y), 5, (0, 255, 0), thickness=-1)
        else:
            print(f"Skipping out-of-bounds keypoint: ({x}, {y})")
        
    # Draw keypoints on the inpainted frame
#     output_frame_with_kp = draw_keypoints(inpainted_frame, keypoints)
    output_frame_with_kp = output_frame
    # Save or display the frame
    out.write(output_frame_with_kp)
    cv2.imshow('Frame with Keypoints', output_frame_with_kp)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press Q to stop
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.nd

In [ ]:
import numpy as np
import cv2

# Create a blank image
test_image = np.zeros((480, 640, 3), dtype=np.uint8)
print(f"Image dtype: {test_image.dtype}, shape: {test_image.shape}")
# Draw a circle on the test image
cv2.circle(test_image, (100, 100), 10, (0, 255, 0), -1)

# Display the test image
cv2.imshow('Test Circle', test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Image dtype: uint8, shape: (480, 640, 3)
